In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix

In [ ]:
imgs = []
imgs.append(cv2.imread("set1/1.jpg"))
imgs.append(cv2.imread("set1/2.jpg"))
imgs.append(cv2.imread("set1/3.jpg"))

In [ ]:
def part2(img, matrix):
    rows,cols,_ = img.shape
    result = cv2.warpPerspective(img, matrix, (1000, 800))
    plt.imshow(result)
    plt.show()
    return result

In [ ]:
rotation_matrix = np.float32([[np.cos(np.radians(10)), -np.sin(np.radians(10)), 0], [np.sin(np.radians(10)), np.cos(np.radians(10)), 0], [0, 0, 1]])
part2(imgs[0], rotation_matrix)

translation_matrix = np.float32([[1, 0, 100], [0, 1, 0], [0, 0, 1]])
part2(imgs[1], translation_matrix)


shrink_matrix = np.float32([[(1/2), 0, 0], [0, (1/2), 0], [0, 0, 1]])
part2(imgs[2], shrink_matrix)

In [ ]:
#  Match features

def match_features1(feature1, feature2, f1, f2):
    distance_M = distance_matrix(feature1, feature2)
    i= 0
    indices = dict()
    height, width = len(feature1), len(feature2)
    while i < height:
        j = 0
        while j < width:
            indices[(i, j)] = distance_M[i][j]
            j += 1
        i += 1

    sorted_map= {k: v for k, v in sorted(indices.items(), key=lambda item: item[1])}

    result1, result2, i = [], [], 0
    for k, v in sorted_map.items():
        result1.append(f1[k[0]])
        result2.append(f2[k[1]])
        if i >= 99: return result1, result2
        i += 1


    return result1, result2

In [ ]:
# Compute SIFT
# Panoramic stitching
sift = cv2.SIFT_create()
# Compute the SIFT features for each image
features, points = [], []
for x in imgs:
    [f, d] = sift.detectAndCompute(x, None)
    features.append(d)
    f = [z.pt for z in f]
    points.append(f)


In [ ]:



# Compute the matching features between jpg1 and jpg2
matched_12_1, matched_12_2 = match_features1(features[0], features[1], points[0], points[1])


# Compute the matching features between jpg1 and jpg3
matched_13_2, matched_13_3 = match_features1(features[1], features[2], points[1], points[2])


# image 1, image 2
M1, mask1 = cv2.findHomography(np.float32(matched_12_1), np.float32(matched_12_2), cv2.RANSAC, 2.0)

# image 3, image 2
M2, mask2 = cv2.findHomography(np.float32(matched_13_3), np.float32(matched_13_2), cv2.RANSAC, 2.0)

translation_matrix1 = np.float32([[1, 0, 350], [0, 1, 300], [0, 0, 1]])
g = np.dot(translation_matrix1, M1)

img1 = part2(imgs[0], g)

g2 = np.dot(translation_matrix1, M2)
img2 = part2(imgs[2], g2)

# translate image 2
img3 = part2(imgs[1], translation_matrix1)

# Fuse the images together
# fuse1 = np.maximum(img1, img3)
# fuse2 = np.maximum(fuse1, img2)
fuse2 = np.maximum(img2, img3)

plt.imshow(fuse2)
plt.show()
